# Pre-processing

In [138]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from datetime import datetime
from datetime import date

In [139]:
#create dataframe
df = pd.read_csv("../Data/pharmacy_tx.csv")
df.head()

,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay
0,2022-01-02,Pharmacy #6,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39
1,2022-01-02,Pharmacy #42,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02
2,2022-01-02,Pharmacy #37,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39
3,2022-01-02,Pharmacy #30,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84
4,2022-01-02,Pharmacy #18,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00


In [140]:
def get_doy(d):
    return d.dayofyear

In [141]:
#convert date to integer between 1 and 365
df['day'] = pd.to_datetime(df.tx_date).apply(get_doy)
df = df.drop(columns = ['tx_date'])
df.head()

,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,day
0,Pharmacy #6,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39,2
1,Pharmacy #42,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02,2
2,Pharmacy #37,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39,2
3,Pharmacy #30,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84,2
4,Pharmacy #18,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00,2


In [142]:
#drop rejected column
df = df[df['rejected']==False]
df = df.drop(columns = ['rejected'])

In [126]:
#drop additional extraneous columns; convert bin to str so that one-hot encoding works correctly
df = df.drop(columns = ['pharmacy', 'group', 'pcn'])
df['bin'] = df['bin'].map(str)
df.head()

,diagnosis,drug,bin,patient_pay,day
0,G99.93,branded tanoclolol,725700,13.39,2
1,U60.52,branded oxasoted,664344,7.02,2
2,Q85.91,branded cupitelol,725700,13.39,2
3,U60.52,generic oxasoted,571569,10.84,2
4,N55.01,branded mamate,664344,47.00,2


In [127]:
#make a column for branded/generic
df['drug_type'] = df['drug'].apply(lambda x: x.split(' ')[0])
df['drug'] = df['drug'].apply(lambda x: x.split(' ')[1])
df.head()

,diagnosis,drug,bin,patient_pay,day,drug_type
0,G99.93,tanoclolol,725700,13.39,2,branded
1,U60.52,oxasoted,664344,7.02,2,branded
2,Q85.91,cupitelol,725700,13.39,2,branded
3,U60.52,oxasoted,571569,10.84,2,generic
4,N55.01,mamate,664344,47.00,2,branded


In [130]:
#separate out X and y
df= df[['day', 'drug_type', 'drug', 'diagnosis', 'bin', 'patient_pay']]
X = df.copy()
y = X.iloc[:,-1]
X = X.drop(columns = ['patient_pay'])

In [132]:
#one-hot encoding
X = pd.get_dummies(X, drop_first = True)

In [133]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [116]:
X_train.head()

,day,pharmacy_Pharmacy #1,pharmacy_Pharmacy #10,pharmacy_Pharmacy #11,pharmacy_Pharmacy #12,pharmacy_Pharmacy #13,pharmacy_Pharmacy #14,pharmacy_Pharmacy #15,pharmacy_Pharmacy #16,pharmacy_Pharmacy #17,...,pcn_W1LW9Y,pcn_W7L3,pcn_WM6A,pcn_XH4T3,pcn_YFVIA,pcn_YICC41,pcn_YL5CMT,pcn_YY6B1J4E8KJ3,pcn_ZQPX,pcn_ZX2QUWR
5197337,146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
643154,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11999316,317,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2548632,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13471059,351,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Random Forest

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [117]:
#fits model on sample_prop proportion of the data n_sample times. Makes predictions on test set, and averages
#to produce a single prediction. Returns KPIs of prediction.

def run_model_samples(model, X_train, y_train, X_test, y_test, n_samples, sample_prop):
    preds = np.zeros((n_samples, len(y_test)))
    for i in range(n_samples):
        #Get random subset
        I = np.random.choice(X_train.index, size = int(sample_prop*len(X_train)), replace = False)

        #Get sub-datasets
        X_tt, y_tt = X_train.loc[I], y_train.loc[I]
        
        # Fit Model
        model.fit(X_tt,y_tt)
    
        # Get Prediction for each sample
        preds[i] = np.array(list(map(relu,model.predict(X_test))))
    
    #Average predictions
    avg_preds = np.mean(preds, axis = 0)
    
    #Calculate KPIs of average predictions
    rmse = np.sqrt(mean_squared_error(y_test, avg_preds))
    print(f'RMSE : {rmse}')
    rmsle = mean_squared_log_error(y_test, avg_preds, squared = False)
    print(f'RMLSE: {rmsle}')
    return avg_preds

In [120]:
def relu(x):
    return max(0,x)

In [56]:
trees = [10,50,100]

for n in trees:
    rfr_model = RandomForestRegressor(n_estimators=n)
    run_model_samples(rfr_model, X_train, y_train, X_test, y_test, 10, 0.1)

RMSE : 15.652256118679052
RMLSE: 0.3432286794310811
RMSE : 15.656991513648718
RMLSE: 0.34302011363041734
RMSE : 15.635791618219661
RMLSE: 0.3429155187041391
